# Torchmeter

An `all-in-one` tool for `Pytorch` model analysis, measuring:
- Params,
- FLOPs / MACs (aka. MACC or MADD), 
- Memory cost, 
- Inference time
- Throughput

Project: https://github.com/Ahzyuan/torchmeter

In [1]:
# install 
# %pip install torchmeter

In [2]:
import torch
from rich import print
from torchvision import models
from torchmeter import Meter, get_config

# 1. 模型准备与Meter初始化
model = models.vgg19_bn()
metered_model = Meter(model)
cfg = get_config()

# 2. 零侵入代理
metered_model.features.requires_grad_(False)

# 3. 输入处理与推理（展示设备同步）
input = torch.randn(1, 3, 224, 224)
if torch.cuda.is_available():
    metered_model.to('cuda')
output = metered_model(input)  # 标准前向传播

Finish Scanning model in 0.0123 seconds

In [3]:
# 4. 模型结构分析
# --------------------------------------------
## 启用重复块折叠
print("="*10, " enable smart folding of repeat blocks", "="*10)
metered_model.tree_fold_repeat = True
print(metered_model.structure)

## 不启用
print("="*10, " disable smart folding of repeat blocks", "="*10)
metered_model.tree_fold_repeat = False
print(metered_model.structure)

==========  enable smart folding of repeat blocks ==========

VGG
├── (1) features Sequential
│   ├── (1.1) 0 Conv2d
│   ├── (1.2) 1 BatchNorm2d
│   ├── (1.3) 2 ReLU
│   ├── (1.4) 3 Conv2d
│   ├── (1.5) 4 BatchNorm2d
│   ├── (1.6) 5 ReLU
│   ├── (1.7) 6 MaxPool2d
│   ├── (1.8) 7 Conv2d
│   ├── (1.9) 8 BatchNorm2d
│   ├── (1.10) 9 ReLU
│   ├── (1.11) 10 Conv2d
│   ├── (1.12) 11 BatchNorm2d
│   ├── (1.13) 12 ReLU
│   ├── (1.14) 13 MaxPool2d
│   ├── (1.15) 14 Conv2d
│   ├── ┏━━━━ Repeat [3] Times ━━━━┓
│   │   ┃ (1.x) 15 BatchNorm2d     ┃
│   │   ┃ (1.(x+1)) 16 ReLU        ┃
│   │   ┃ (1.(x+2)) 17 Conv2d      ┃
│   │   ┃ ------------------------ ┃
│   │   ┃ Where x = 16, 19, 22     ┃
│   │   ┗━━━━━━━━━━━━━━━━━━━━━━━━━━┛
│   ├── (1.25) 24 BatchNorm2d
│   ├── (1.26) 25 ReLU
│   ├── (1.27) 26 MaxPool2d
│   ├── (1.28) 27 Conv2d
│   ├── ┏━━━━ Repeat [3] Times ━━━━┓
│   │   ┃ (1.y) 28 BatchNorm2d     ┃
│   │   ┃ (1.(y+1)) 29 ReLU        ┃
│   │   ┃ (1.(y+2)) 30 Conv2d      ┃
│   │   ┃ ------------------------ ┃
│   │   ┃ Where y = 29, 32, 35     ┃
│   │   ┗━━━━━━━━━━━━━━━━━━━━━━━━━━┛
│   ├── (1.38) 37 BatchNorm2d
│   ├── (1.39) 38 ReLU
│   ├── (1.40) 39 MaxPool2d
│   ├── ┏━━━━━━ Repeat [2] Times ━━━━━━┓
│   │   ┃ (1.i) 40 Conv2d              ┃
│   │   ┃ (1.(i+1)) 41 BatchNorm2d     ┃
│   │   ┃ (1.(i+2)) 42 ReLU            ┃
│   │   ┃ (1.(i+3)) 43 Conv2d          ┃
│   │   ┃ (1.(i+4)) 44 BatchNorm2d     ┃
│   │   ┃ (1.(i+5)) 45 ReLU            ┃
│   │   ┃ ---------------------------- ┃
│   │   ┃ Where i = 41, 47             ┃
│   │   ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
│   └── (1.53) 52 MaxPool2d
├── (2) avgpool AdaptiveAvgPool2d
└── (3) classifier Sequential
    ├── (3.1) 0 Linear
    ├── (3.2) 1 ReLU
    ├── (3.3) 2 Dropout
    ├── (3.4) 3 Linear
    ├── (3.5) 4 ReLU
    ├── (3.6) 5 Dropout
    └── (3.7) 6 Linear

==========  disable smart folding of repeat blocks ==========

VGG
├── (1) features Sequential
│   ├── (1.1) 0 Conv2d
│   ├── (1.2) 1 BatchNorm2d
│   ├── (1.3) 2 ReLU
│   ├── (1.4) 3 Conv2d
│   ├── (1.5) 4 BatchNorm2d
│   ├── (1.6) 5 ReLU
│   ├── (1.7) 6 MaxPool2d
│   ├── (1.8) 7 Conv2d
│   ├── (1.9) 8 BatchNorm2d
│   ├── (1.10) 9 ReLU
│   ├── (1.11) 10 Conv2d
│   ├── (1.12) 11 BatchNorm2d
│   ├── (1.13) 12 ReLU
│   ├── (1.14) 13 MaxPool2d
│   ├── (1.15) 14 Conv2d
│   ├── (1.16) 15 BatchNorm2d
│   ├── (1.17) 16 ReLU
│   ├── (1.18) 17 Conv2d
│   ├── (1.19) 18 BatchNorm2d
│   ├── (1.20) 19 ReLU
│   ├── (1.21) 20 Conv2d
│   ├── (1.22) 21 BatchNorm2d
│   ├── (1.23) 22 ReLU
│   ├── (1.24) 23 Conv2d
│   ├── (1.25) 24 BatchNorm2d
│   ├── (1.26) 25 ReLU
│   ├── (1.27) 26 MaxPool2d
│   ├── (1.28) 27 Conv2d
│   ├── (1.29) 28 BatchNorm2d
│   ├── (1.30) 29 ReLU
│   ├── (1.31) 30 Conv2d
│   ├── (1.32) 31 BatchNorm2d
│   ├── (1.33) 32 ReLU
│   ├── (1.34) 33 Conv2d
│   ├── (1.35) 34 BatchNorm2d
│   ├── (1.36) 35 ReLU
│   ├── (1.37) 36 Conv2d
│   ├── (1.38) 37 BatchNorm2d
│   ├── (1.39) 38 ReLU
│   ├── (1.40) 39 MaxPool2d
│   ├── (1.41) 40 Conv2d
│   ├── (1.42) 41 BatchNorm2d
│   ├── (1.43) 42 ReLU
│   ├── (1.44) 43 Conv2d
│   ├── (1.45) 44 BatchNorm2d
│   ├── (1.46) 45 ReLU
│   ├── (1.47) 46 Conv2d
│   ├── (1.48) 47 BatchNorm2d
│   ├── (1.49) 48 ReLU
│   ├── (1.50) 49 Conv2d
│   ├── (1.51) 50 BatchNorm2d
│   ├── (1.52) 51 ReLU
│   └── (1.53) 52 MaxPool2d
├── (2) avgpool AdaptiveAvgPool2d
└── (3) classifier Sequential
    ├── (3.1) 0 Linear
    ├── (3.2) 1 ReLU
    ├── (3.3) 2 Dropout
    ├── (3.4) 3 Linear
    ├── (3.5) 4 ReLU
    ├── (3.6) 5 Dropout
    └── (3.7) 6 Linear

In [4]:
# 5. 性能分析（展示全栈分析能力）
cfg.render_interval = 0
# --------------------------------------------
# 5.1 参数分析（展示参数测量）
print("="*10, " Parameter Analysis ", "="*10)
print(metered_model.param)  # 总参数统计
tb, data = metered_model.profile('param', no_tree=True)  # 分层参数分布

==========  Parameter Analysis  ==========

Params_INFO
•     Operation_Id = 0
•   Operation_Name = VGG
•   Operation_Type = VGG
•     Total_Params = 143678248.00 = 143.68 M
• Learnable_Params = 123642856.00 = 123.64 M

╭──────────────┬────────────────┬───────────────────┬────────────┬───────────────┬─────────────╮
│ Operation_Id │ Operation_Name │  Operation_Type   │ Param_Name │ Requires_Grad │ Numeric_Num │
├──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────┤
│      1       │    features    │    Sequential     │     -      │       -       │    0.00     │
│     1.1      │       0        │      Conv2d       │   weight   │     False     │   1.73 K    │
│     1.1      │       0        │      Conv2d       │    bias    │     False     │    64.00    │
│     1.2      │       1        │    BatchNorm2d    │   weight   │     False     │    64.00    │
│     1.2      │       1        │    BatchNorm2d    │    bias    │     False     │    64.00    │
│     1.3      │       2        │       ReLU        │     -      │       -       │    0.00     │
│     1.4      │       3        │      Conv2d       │   weight   │     False     │   36.86 K   │
│     1.4      │       3        │      Conv2d       │    bias    │     False     │    64.00    │
│     1.5      │       4        │    BatchNorm2d    │   weight   │     False     │    64.00    │
│     1.5      │       4        │    BatchNorm2d    │    bias    │     False     │    64.00    │
│     1.6      │       5        │       ReLU        │     -      │       -       │    0.00     │
│     1.7      │       6        │     MaxPool2d     │     -      │       -       │    0.00     │
│     1.8      │       7        │      Conv2d       │   weight   │     False     │   73.73 K   │
│     1.8      │       7        │      Conv2d       │    bias    │     False     │   128.00    │
│     1.9      │       8        │    BatchNorm2d    │   weight   │     False     │   128.00    │
│     1.9      │       8        │    BatchNorm2d    │    bias    │     False     │   128.00    │
│     1.10     │       9        │       ReLU        │     -      │       -       │    0.00     │
│     1.11     │       10       │      Conv2d       │   weight   │     False     │  147.46 K   │
│     1.11     │       10       │      Conv2d       │    bias    │     False     │   128.00    │
│     1.12     │       11       │    BatchNorm2d    │   weight   │     False     │   128.00    │
│     1.12     │       11       │    BatchNorm2d    │    bias    │     False     │   128.00    │
│     1.13     │       12       │       ReLU        │     -      │       -       │    0.00     │
│     1.14     │       13       │     MaxPool2d     │     -      │       -       │    0.00     │
│     1.15     │       14       │      Conv2d       │   weight   │     False     │  294.91 K   │
│     1.15     │       14       │      Conv2d       │    bias    │     False     │   256.00    │
│     1.16     │       15       │    BatchNorm2d    │   weight   │     False     │   256.00    │
│     1.16     │       15       │    BatchNorm2d    │    bias    │     False     │   256.00    │
│     1.17     │       16       │       ReLU        │     -      │       -       │    0.00     │
│     1.18     │       17       │      Conv2d       │   weight   │     False     │  589.82 K   │
│     1.18     │       17       │      Conv2d       │    bias    │     False     │   256.00    │
│     1.19     │       18       │    BatchNorm2d    │   weight   │     False     │   256.00    │
│     1.19     │       18       │    BatchNorm2d    │    bias    │     False     │   256.00    │
│     1.20     │       19       │       ReLU        │     -      │       -       │    0.00     │
│     1.21     │       20       │      Conv2d       │   weight   │     False     │  589.82 K   │
│     1.21     │       20       │      Conv2d       │    bias    │     False     │   256.00    │
│     1.22     │       21       │    BatchNorm2d    │   weight   │     False     │   256.00    │
│     1.22     │       21       │    BatchNorm2d    │    bias    │     False     │   256.00    │
│     1.23     │       22       │       ReLU        │     -      │       -       │    0.00     │
│     1.24     │       

In [5]:
# 5.2 计算分析（展示FLOPs/MACs）
print("="*10, " Computational Profiling ", "="*10)
print(metered_model.cal)
tb, data = metered_model.profile('cal', no_tree=True)

==========  Computational Profiling  ==========

Calculation_INFO
•   Operation_Id = 0
• Operation_Type = VGG
• Operation_Name = VGG
•           MACs = 19681218048.00 = 19.68 G
•          FLOPs = 39342984704.00 = 39.34 G

╭────────────┬────────────┬────────────┬────────────┬──────┬────────────┬────────────┬─────────────┬────────────╮
│ Operation_ │ Operation_ │ Operation_ │ Kernel_Siz │      │            │            │             │            │
│     Id     │    Name    │    Type    │     e      │ Bias │   Input    │   Output   │    MACs     │   FLOPs    │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│     1      │  features  │ Sequential │     -      │  -   │   [1, 3,   │  [1, 512,  │   19.56 G   │  39.10 G   │
│            │            │            │            │      │ 224, 224]  │   7, 7]    │             │            │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│    1.1     │     0      │   Conv2d   │   [3, 3]   │ True │   [1, 3,   │  [1, 64,   │   86.70 M   │  173.41 M  │
│            │            │            │            │      │ 224, 224]  │ 224, 224]  │             │            │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│    1.2     │     1      │ BatchNorm2 │     -      │  -   │  [1, 64,   │  [1, 64,   │   6.42 M    │  12.85 M   │
│            │            │     d      │            │      │ 224, 224]  │ 224, 224]  │             │            │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│    1.3     │     2      │    ReLU    │     -      │  -   │  [1, 64,   │  [1, 64,   │   3.21 M    │   3.21 M   │
│            │            │            │            │      │ 224, 224]  │ 224, 224]  │             │            │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│    1.4     │     3      │   Conv2d   │   [3, 3]   │ True │  [1, 64,   │  [1, 64,   │   1.85 G    │   3.70 G   │
│            │            │            │            │      │ 224, 224]  │ 224, 224]  │             │            │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│    1.5     │     4      │ BatchNorm2 │     -      │  -   │  [1, 64,   │  [1, 64,   │   6.42 M    │  12.85 M   │
│            │            │     d      │            │      │ 224, 224]  │ 224, 224]  │             │            │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│    1.6     │     5      │    ReLU    │     -      │  -   │  [1, 64,   │  [1, 64,   │   3.21 M    │   3.21 M   │
│            │            │            │            │      │ 224, 224]  │ 224, 224]  │             │            │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│    1.7     │     6      │ MaxPool2d  │   [2, 2]   │  -   │  [1, 64,   │  [1, 64,   │   2.41 M    │   2.41 M   │
│            │            │            │            │      │ 224, 224]  │ 112, 112]  │             │            │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│    1.8     │     7      │   Conv2d   │   [3, 3]   │ True │  [1, 64,   │  [1, 128,  │  924.84 M   │   1.85 G   │
│            │            │            │            │      │ 112, 112]  │ 112, 112]  │             │            │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│    1.9     │     8      │ BatchNorm2 │     -      │  -   │  [1, 128,  │  [1, 128,  │   3.21 M    │   6.42 M   │
│            │            │     d      │            │      │ 112, 112]  │ 112, 112]  │             │            │
├────────────┼────────────┼────────────┼────────────┼──────┼────────────┼────────────┼─────────────┼────────────┤
│    1.10    │     9      │    ReLU    │     -      │  -   │  [1, 128,  │  [1, 128,  │   1.61 M    │   1.61 M   │
│         

In [6]:
# 5.3 内存分析（展示内存诊断）
print("="*10, " Memory Diagnostics ", "="*10)
print(metered_model.mem)
tb, data = metered_model.profile('mem', no_tree=True)

==========  Memory Diagnostics  ==========

Memory_INFO
•   Operation_Id = 0
• Operation_Type = VGG
• Operation_Name = VGG
•     Param_Cost = 574712992.00 = 548.09 MiB
•    Buffer_Cost = 44160.00 = 43.12 KiB
•    Output_Cost = 125108128.00 = 119.31 MiB
•          Total = 699865280.00 = 667.44 MiB

╭──────────────┬────────────────┬───────────────────┬────────────┬─────────────┬─────────────┬────────────╮
│ Operation_Id │ Operation_Name │  Operation_Type   │ Param_Cost │ Buffer_Cost │ Output_Cost │   Total    │
├──────────────┼────────────────┼───────────────────┼────────────┼─────────────┼─────────────┼────────────┤
│      1       │    features    │    Sequential     │ 76.43 MiB  │  43.12 KiB  │ 119.15 MiB  │ 195.62 MiB │
│     1.1      │       0        │      Conv2d       │   7 KiB    │      -      │  12.25 MiB  │ 12.26 MiB  │
│     1.2      │       1        │    BatchNorm2d    │   512 B    │    520 B    │  12.25 MiB  │ 12.25 MiB  │
│     1.3      │       2        │   ReLU(inplace)   │     -      │      -      │      -      │     -      │
│     1.4      │       3        │      Conv2d       │ 144.25 KiB │      -      │  12.25 MiB  │ 12.39 MiB  │
│     1.5      │       4        │    BatchNorm2d    │   512 B    │    520 B    │  12.25 MiB  │ 12.25 MiB  │
│     1.6      │       5        │   ReLU(inplace)   │     -      │      -      │      -      │     -      │
│     1.7      │       6        │     MaxPool2d     │     -      │      -      │  3.06 MiB   │  3.06 MiB  │
│     1.8      │       7        │      Conv2d       │ 288.50 KiB │      -      │  6.12 MiB   │  6.41 MiB  │
│     1.9      │       8        │    BatchNorm2d    │   1 KiB    │  1.01 KiB   │  6.12 MiB   │  6.13 MiB  │
│     1.10     │       9        │   ReLU(inplace)   │     -      │      -      │      -      │     -      │
│     1.11     │       10       │      Conv2d       │ 576.50 KiB │      -      │  6.12 MiB   │  6.69 MiB  │
│     1.12     │       11       │    BatchNorm2d    │   1 KiB    │  1.01 KiB   │  6.12 MiB   │  6.13 MiB  │
│     1.13     │       12       │   ReLU(inplace)   │     -      │      -      │      -      │     -      │
│     1.14     │       13       │     MaxPool2d     │     -      │      -      │  1.53 MiB   │  1.53 MiB  │
│     1.15     │       14       │      Conv2d       │  1.13 MiB  │      -      │  3.06 MiB   │  4.19 MiB  │
│     1.16     │       15       │    BatchNorm2d    │   2 KiB    │  2.01 KiB   │  3.06 MiB   │  3.07 MiB  │
│     1.17     │       16       │   ReLU(inplace)   │     -      │      -      │      -      │     -      │
│     1.18     │       17       │      Conv2d       │  2.25 MiB  │      -      │  3.06 MiB   │  5.31 MiB  │
│     1.19     │       18       │    BatchNorm2d    │   2 KiB    │  2.01 KiB   │  3.06 MiB   │  3.07 MiB  │
│     1.20     │       19       │   ReLU(inplace)   │     -      │      -      │      -      │     -      │
│     1.21     │       20       │      Conv2d       │  2.25 MiB  │      -      │  3.06 MiB   │  5.31 MiB  │
│     1.22     │       21       │    BatchNorm2d    │   2 KiB    │  2.01 KiB   │  3.06 MiB   │  3.07 MiB  │
│     1.23     │       22       │   ReLU(inplace)   │     -      │      -      │      -      │     -      │
│     1.24     │       23       │      Conv2d       │  2.25 MiB  │      -      │  3.06 MiB   │  5.31 MiB  │
│     1.25     │       24       │    BatchNorm2d    │   2 KiB    │  2.01 KiB   │  3.06 MiB   │  3.07 MiB  │
│     1.26     │       25       │   ReLU(inplace)   │     -      │      -      │      -      │     -      │
│     1.27     │       26       │     MaxPool2d     │     -      │      -      │   784 KiB   │  784 KiB   │
│     1.28     │       27       │      Conv2d       │  4.50 MiB  │      -      │  1.53 MiB   │  6.03 MiB  │
│     1.29     │       28       │    BatchNorm2d    │   4 KiB    │  4.01 KiB   │  1.53 MiB   │  1.54 MiB  │
│     1.30     │       29       │   ReLU(inplace)   │     -      │      -      │      -      │     -      │
│     1.31     │       30       │      Conv2d       │  9.00 MiB  │      -      │  1.53 MiB   │ 10.53 MiB  │
│     1.32     │       31       │    BatchNorm2d    │   4 KiB    │  4.01 KiB   │  1.53 MiB   │  1.54 MiB  │
│     1.33     │       32       │   ReLU(inplace)   │     -      │      -      │      -      │     -      │
│   

In [7]:
# 5.4 性能基准测试（展示延迟和吞吐量）
print("="*10, " Inference latency & Throughput benchmarking ", "="*10)
metered_model.ittp_warmup = 10  # 定制预热次数
metered_model.ittp_benchmark_time = 20
print(metered_model.ittp)
tb, data = metered_model.profile('ittp', no_tree=True)

==========  Inference latency & Throughput benchmarking  ==========

Benchmark Inference Time & Throughput: 100%|██████████| 1280/1280 [00:00<00:00, 5445.62module/s]


InferTime_Throughput_INFO
•   Operation_Id = 0
• Operation_Name = VGG
• Operation_Type = VGG
•     Infer_Time = 2.39 ms ± 121.79 us
•     Throughput = 418.86 IPS ± 22.32 IPS

Benchmark Inference Time & Throughput: 100%|██████████| 1280/1280 [00:00<00:00, 4705.27module/s]


╭──────────────┬────────────────┬───────────────────┬───────────────────────┬─────────────────────────╮
│ Operation_Id │ Operation_Name │  Operation_Type   │      Infer_Time       │       Throughput        │
├──────────────┼────────────────┼───────────────────┼───────────────────────┼─────────────────────────┤
│      1       │    features    │    Sequential     │  2.05 ms ± 17.46 us   │  486.78 IPS ± 4.13 IPS  │
│     1.1      │       0        │      Conv2d       │  83.90 us ± 2.39 us   │ 11.92 KIPS ± 337.38 IPS │
│     1.2      │       1        │    BatchNorm2d    │  65.52 us ± 1.16 us   │ 15.26 KIPS ± 273.80 IPS │
│     1.3      │       2        │       ReLU        │ 29.68 us ± 240.00 ns  │ 33.69 KIPS ± 271.93 IPS │
│     1.4      │       3        │      Conv2d       │  130.90 us ± 1.02 us  │  7.64 KIPS ± 60.07 IPS  │
│     1.5      │       4        │    BatchNorm2d    │  64.66 us ± 1.02 us   │ 15.47 KIPS ± 242.20 IPS │
│     1.6      │       5        │       ReLU        │  29.30 us ± 1.06 us   │ 34.14 KIPS ± 1.24 KIPS  │
│     1.7      │       6        │     MaxPool2d     │  39.90 us ± 1.73 us   │ 25.06 KIPS ± 1.09 KIPS  │
│     1.8      │       7        │      Conv2d       │ 94.46 us ± 928.00 ns  │ 10.59 KIPS ± 103.53 IPS │
│     1.9      │       8        │    BatchNorm2d    │  64.54 us ± 1.74 us   │ 15.49 KIPS ± 415.50 IPS │
│     1.10     │       9        │       ReLU        │  29.36 us ± 1.02 us   │ 34.06 KIPS ± 1.19 KIPS  │
│     1.11     │       10       │      Conv2d       │  120.75 us ± 1.26 us  │  8.28 KIPS ± 86.53 IPS  │
│     1.12     │       11       │    BatchNorm2d    │  64.74 us ± 1.10 us   │ 15.45 KIPS ± 262.93 IPS │
│     1.13     │       12       │       ReLU        │ 29.52 us ± 960.00 ns  │ 33.88 KIPS ± 1.12 KIPS  │
│     1.14     │       13       │     MaxPool2d     │  39.94 us ± 1.07 us   │ 25.04 KIPS ± 678.10 IPS │
│     1.15     │       14       │      Conv2d       │  92.66 us ± 2.05 us   │ 10.79 KIPS ± 235.88 IPS │
│     1.16     │       15       │    BatchNorm2d    │  64.51 us ± 1.46 us   │ 15.50 KIPS ± 347.06 IPS │
│     1.17     │       16       │       ReLU        │ 29.47 us ± 840.00 ns  │ 33.93 KIPS ± 980.28 IPS │
│     1.18     │       17       │      Conv2d       │ 117.76 us ± 144.00 ns │  8.49 KIPS ± 10.41 IPS  │
│     1.19     │       18       │    BatchNorm2d    │  64.58 us ± 1.80 us   │ 15.49 KIPS ± 430.95 IPS │
│     1.20     │       19       │       ReLU        │  29.70 us ± 1.23 us   │ 33.67 KIPS ± 1.36 KIPS  │
│     1.21     │       20       │      Conv2d       │  117.76 us ± 1.00 us  │  8.49 KIPS ± 71.71 IPS  │
│     1.22     │       21       │    BatchNorm2d    │  64.58 us ± 1.02 us   │ 15.49 KIPS ± 240.46 IPS │
│     1.23     │       22       │       ReLU        │  29.68 us ± 1.14 us   │ 33.69 KIPS ± 1.28 KIPS  │
│     1.24     │       23       │      Conv2d       │  114.66 us ± 3.78 us  │ 8.72 KIPS ± 281.32 IPS  │
│     1.25     │       24       │    BatchNorm2d    │  64.74 us ± 1.16 us   │ 15.45 KIPS ± 274.97 IPS │
│     1.26     │       25       │       ReLU        │ 29.58 us ± 752.00 ns  │ 33.80 KIPS ± 872.54 IPS │
│     1.27     │       26       │     MaxPool2d     │  38.96 us ± 1.05 us   │ 25.67 KIPS ± 680.30 IPS │
│     1.28     │       27       │      Conv2d       │  92.16 us ± 1.91 us   │ 10.85 KIPS ± 224.32 IPS │
│     1.29     │       28       │    BatchNorm2d    │  63.79 us ± 1.69 us   │ 15.68 KIPS ± 406.97 IPS │
│     1.30     │       29       │       ReLU        │  29.70 us ± 1.33 us   │ 33.67 KIPS ± 1.47 KIPS  │
│     1.31     │       30       │      Conv2d       │ 113.66 us ± 816.00 ns │  8.80 KIPS ± 62.75 IPS  │
│     1.32     │       31       │    BatchNorm2d    │  64.43 us ± 1.73 us   │ 15.52 KIPS ± 417.28 IPS │
│     1.33     │       32       │       ReLU        │ 29.98 us ± 904.00 ns  │ 33.35 KIPS ± 994.66 IPS │
│     1.34     │       33       │      Conv2d       │ 113.74 us ± 720.00 ns │  8.79 KIPS ± 55.37 IPS  │
│     1.35     │       34       │    BatchNorm2d

In [8]:
# 6. 可视化功能（展示实时渲染与配置）
# --------------------------------------------
# 6.1 结构树渲染自定义
from rich.box import ROUNDED

print("="*10, " Tree rendering customization ", "="*10)
metered_model.tree_fold_repeat = True
metered_model.tree_levels_args = {
    "default": {"label": "[b gray35](<node_id>) [green]<name>[/green] [cyan]<module_repr>[/]"},
    "1": {"guide_style": "cornflower_blue"}
}
metered_model.tree_repeat_block_args = {
    "title": "[[b]<repeat_time>[/b]] [i]Times Repeated[/]",
    "box": ROUNDED
}
print(metered_model.structure)

==========  Tree rendering customization  ==========

VGG
├── (1) features Sequential
│   ├── (1.1) 0 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
│   ├── (1.2) 1 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
│   ├── (1.3) 2 ReLU(inplace=True)
│   ├── (1.4) 3 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
│   ├── (1.5) 4 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
│   ├── (1.6) 5 ReLU(inplace=True)
│   ├── (1.7) 6 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
│   ├── (1.8) 7 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
│   ├── (1.9) 8 BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
│   ├── (1.10) 9 ReLU(inplace=True)
│   ├── (1.11) 10 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
│   ├── (1.12) 11 BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
│   ├── (1.13) 12 ReLU(inplace=True)
│   ├── (1.14) 13 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
│   ├── (1.15) 14 Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
│   ├── ╭───────────────────────────────────── [3] Times Repeated ──────────────────────────────────────╮
│   │   │ (1.j) 15 BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)     │
│   │   │ (1.(j+1)) 16 ReLU(inplace=True)                                                               │
│   │   │ (1.(j+2)) 17 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))              │
│   │   │ --------------------------------------------------------------------------------------------- │
│   │   │ Where j = 16, 19, 22                                                                          │
│   │   ╰───────────────────────────────────────────────────────────────────────────────────────────────╯
│   ├── (1.25) 24 BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
│   ├── (1.26) 25 ReLU(inplace=True)
│   ├── (1.27) 26 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
│   ├── (1.28) 27 Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
│   ├── ╭───────────────────────────────────── [3] Times Repeated ──────────────────────────────────────╮
│   │   │ (1.k) 28 BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)     │
│   │   │ (1.(k+1)) 29 ReLU(inplace=True)                                                               │
│   │   │ (1.(k+2)) 30 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))              │
│   │   │ --------------------------------------------------------------------------------------------- │
│   │   │ Where k = 29, 32, 35                                                                          │
│   │   ╰───────────────────────────────────────────────────────────────────────────────────────────────╯
│   ├── (1.38) 37 BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
│   ├── (1.39) 38 ReLU(inplace=True)
│   ├── (1.40) 39 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
│   ├── ╭─────────────────────────────────────── [2] Times Repeated ────────────────────────────────────────╮
│   │   │ (1.a) 40 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))                      │
│   │   │ (1.(a+1)) 41 BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)     │
│   │   │ (1.(a+2)) 42 ReLU(inplace=True)                                                                   │
│   │   │ (1.(a+3)) 43 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))                  │
│   │   │ (1.(a+4)) 44 BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)     │
│   │   │ (1.(a+5)) 45 ReLU(inplace=True)                                                                   │
│   │   │ ---------------------------

In [9]:
# 6.2 表格渲染自定义
print("="*10, " Table rendering customization ", "="*10)
metered_model.table_column_args.justify = "left"
metered_model.table_display_args = {
    "style": "#af8700", # or rgb(175,135,0)
    "show_lines": True,
    "show_edge": False    
}

tb, data = metered_model.profile("param", no_tree=True)

==========  Table rendering customization  ==========

 Operation_Id │ Operation_Name │ Operation_Type    │ Param_Name │ Requires_Grad │ Numeric_Num 
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1            │ features       │ Sequential        │ -          │ -             │ 0.00        
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.1          │ 0              │ Conv2d            │ weight     │ False         │ 1.73 K      
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.1          │ 0              │ Conv2d            │ bias       │ False         │ 64.00       
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.2          │ 1              │ BatchNorm2d       │ weight     │ False         │ 64.00       
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.2          │ 1              │ BatchNorm2d       │ bias       │ False         │ 64.00       
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.3          │ 2              │ ReLU              │ -          │ -             │ 0.00        
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.4          │ 3              │ Conv2d            │ weight     │ False         │ 36.86 K     
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.4          │ 3              │ Conv2d            │ bias       │ False         │ 64.00       
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.5          │ 4              │ BatchNorm2d       │ weight     │ False         │ 64.00       
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.5          │ 4              │ BatchNorm2d       │ bias       │ False         │ 64.00       
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.6          │ 5              │ ReLU              │ -          │ -             │ 0.00        
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.7          │ 6              │ MaxPool2d         │ -          │ -             │ 0.00        
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.8          │ 7              │ Conv2d            │ weight     │ False         │ 73.73 K     
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.8          │ 7              │ Conv2d            │ bias       │ False         │ 128.00      
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.9          │ 8              │ BatchNorm2d       │ weight     │ False         │ 128.00      
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.9          │ 8              │ BatchNorm2d       │ bias       │ False         │ 128.00      
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.10         │ 9              │ ReLU              │ -          │ -             │ 0.00        
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.11         │ 10             │ Conv2d            │ weight     │ False         │ 147.46 K    
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.11         │ 10             │ Conv2d            │ bias       │ False         │ 128.00      
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.12         │ 11             │ BatchNorm2d       │ weight     │ False         │ 128.00      
──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────
 1.12     

In [10]:
# 6.2 表格内容渲染自定义
# 6.2.1 树 + 表
print("="*10, " Table report with tree ", "="*10)
cfg.restore()
cfg.render_interval = 0
tb, data = metered_model.profile("param", 
                                 no_tree=False)

==========  Table report with tree  ==========

VGG                                       ╭───────────┬───────────┬───────────┬───────────┬───────────┬───────────╮
├── (1) features Sequential               │ Operation │ Operation │ Operation │ Param_Nam │ Requires_ │ Numeric_N │
│   ├── (1.1) 0 Conv2d                    │    _Id    │   _Name   │   _Type   │     e     │   Grad    │    um     │
│   ├── (1.2) 1 BatchNorm2d               ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   ├── (1.3) 2 ReLU                      │     1     │ features  │ Sequentia │     -     │     -     │   0.00    │
│   ├── (1.4) 3 Conv2d                    │           │           │     l     │           │           │           │
│   ├── (1.5) 4 BatchNorm2d               ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   ├── (1.6) 5 ReLU                      │    1.1    │     0     │  Conv2d   │  weight   │   False   │  1.73 K   │
│   ├── (1.7) 6 MaxPool2d                 ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   ├── (1.8) 7 Conv2d                    │    1.1    │     0     │  Conv2d   │   bias    │   False   │   64.00   │
│   ├── (1.9) 8 BatchNorm2d               ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   ├── (1.10) 9 ReLU                     │    1.2    │     1     │ BatchNorm │  weight   │   False   │   64.00   │
│   ├── (1.11) 10 Conv2d                  │           │           │    2d     │           │           │           │
│   ├── (1.12) 11 BatchNorm2d             ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   ├── (1.13) 12 ReLU                    │    1.2    │     1     │ BatchNorm │   bias    │   False   │   64.00   │
│   ├── (1.14) 13 MaxPool2d               │           │           │    2d     │           │           │           │
│   ├── (1.15) 14 Conv2d                  ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   ├── ┏━━━━ Repeat [3] Times ━━━━┓      │    1.3    │     2     │   ReLU    │     -     │     -     │   0.00    │
│   │   ┃ (1.b) 15 BatchNorm2d     ┃      ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   │   ┃ (1.(b+1)) 16 ReLU        ┃      │    1.4    │     3     │  Conv2d   │  weight   │   False   │  36.86 K  │
│   │   ┃ (1.(b+2)) 17 Conv2d      ┃      ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   │   ┃ ------------------------ ┃      │    1.4    │     3     │  Conv2d   │   bias    │   False   │   64.00   │
│   │   ┃ Where b = 16, 19, 22     ┃      ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   │   ┗━━━━━━━━━━━━━━━━━━━━━━━━━━┛      │    1.5    │     4     │ BatchNorm │  weight   │   False   │   64.00   │
│   ├── (1.25) 24 BatchNorm2d             │           │           │    2d     │           │           │           │
│   ├── (1.26) 25 ReLU                    ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   ├── (1.27) 26 MaxPool2d               │    1.5    │     4     │ BatchNorm │   bias    │   False   │   64.00   │
│   ├── (1.28) 27 Conv2d                  │           │           │    2d     │           │           │           │
│   ├── ┏━━━━ Repeat [3] Times ━━━━┓      ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   │   ┃ (1.c) 28 BatchNorm2d     ┃      │    1.6    │     5     │   ReLU    │     -     │     -     │   0.00    │
│   │   ┃ (1.(c+1)) 29 ReLU        ┃      ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   │   ┃ (1.(c+2)) 30 Conv2d      ┃      │    1.7    │     6     │ MaxPool2d │     -     │     -     │   0.00    │
│   │   ┃ ------------------------ ┃      ├───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│   │   ┃ Where c = 29, 32, 35     ┃      │    1.8    │     7     │  Conv2d   │  weight   │   False   │  73.73 K  │
│   │   ┗━━━━━━━━━━━━━━━━━━━━━━━━━━┛      ├───────────┼─

In [11]:
# 6.2.2 树 + 表
print("="*10, " Table report with raw data ", "="*10)
tb, data = metered_model.profile("param", no_tree=True, raw_data=True)

==========  Table report with raw data  ==========

╭──────────────┬────────────────┬───────────────────┬────────────┬───────────────┬─────────────╮
│ Operation_Id │ Operation_Name │  Operation_Type   │ Param_Name │ Requires_Grad │ Numeric_Num │
├──────────────┼────────────────┼───────────────────┼────────────┼───────────────┼─────────────┤
│      1       │    features    │    Sequential     │     -      │       -       │     0.0     │
│     1.1      │       0        │      Conv2d       │   weight   │     False     │   1728.0    │
│     1.1      │       0        │      Conv2d       │    bias    │     False     │    64.0     │
│     1.2      │       1        │    BatchNorm2d    │   weight   │     False     │    64.0     │
│     1.2      │       1        │    BatchNorm2d    │    bias    │     False     │    64.0     │
│     1.3      │       2        │       ReLU        │     -      │       -       │     0.0     │
│     1.4      │       3        │      Conv2d       │   weight   │     False     │   36864.0   │
│     1.4      │       3        │      Conv2d       │    bias    │     False     │    64.0     │
│     1.5      │       4        │    BatchNorm2d    │   weight   │     False     │    64.0     │
│     1.5      │       4        │    BatchNorm2d    │    bias    │     False     │    64.0     │
│     1.6      │       5        │       ReLU        │     -      │       -       │     0.0     │
│     1.7      │       6        │     MaxPool2d     │     -      │       -       │     0.0     │
│     1.8      │       7        │      Conv2d       │   weight   │     False     │   73728.0   │
│     1.8      │       7        │      Conv2d       │    bias    │     False     │    128.0    │
│     1.9      │       8        │    BatchNorm2d    │   weight   │     False     │    128.0    │
│     1.9      │       8        │    BatchNorm2d    │    bias    │     False     │    128.0    │
│     1.10     │       9        │       ReLU        │     -      │       -       │     0.0     │
│     1.11     │       10       │      Conv2d       │   weight   │     False     │  147456.0   │
│     1.11     │       10       │      Conv2d       │    bias    │     False     │    128.0    │
│     1.12     │       11       │    BatchNorm2d    │   weight   │     False     │    128.0    │
│     1.12     │       11       │    BatchNorm2d    │    bias    │     False     │    128.0    │
│     1.13     │       12       │       ReLU        │     -      │       -       │     0.0     │
│     1.14     │       13       │     MaxPool2d     │     -      │       -       │     0.0     │
│     1.15     │       14       │      Conv2d       │   weight   │     False     │  294912.0   │
│     1.15     │       14       │      Conv2d       │    bias    │     False     │    256.0    │
│     1.16     │       15       │    BatchNorm2d    │   weight   │     False     │    256.0    │
│     1.16     │       15       │    BatchNorm2d    │    bias    │     False     │    256.0    │
│     1.17     │       16       │       ReLU        │     -      │       -       │     0.0     │
│     1.18     │       17       │      Conv2d       │   weight   │     False     │  589824.0   │
│     1.18     │       17       │      Conv2d       │    bias    │     False     │    256.0    │
│     1.19     │       18       │    BatchNorm2d    │   weight   │     False     │    256.0    │
│     1.19     │       18       │    BatchNorm2d    │    bias    │     False     │    256.0    │
│     1.20     │       19       │       ReLU        │     -      │       -       │     0.0     │
│     1.21     │       20       │      Conv2d       │   weight   │     False     │  589824.0   │
│     1.21     │       20       │      Conv2d       │    bias    │     False     │    256.0    │
│     1.22     │       21       │    BatchNorm2d    │   weight   │     False     │    256.0    │
│     1.22     │       21       │    BatchNorm2d    │    bias    │     False     │    256.0    │
│     1.23     │       22       │       ReLU        │     -      │       -       │     0.0     │
│     1.24     │       

In [12]:
# 6.2.3 表格列自定义
print("="*10, " Table structure customization ", "="*10)
tb, data = metered_model.profile(
    "mem", 
    no_tree=True, 
    pick_cols=["Operation_Id", "Operation_Name", "Param_Cost", "Buffer_Cost", "Output_Cost", "Total"], 
    exclude_cols=["Operation_Name"],
    custom_cols={"Operation_Id": "ID", 
                 "Param_Cost": "Param Cost", 
                 "Buffer_Cost": "Buffer Cost", 
                 "Output_Cost": "Output Cost"},
    keep_custom_name = True,
    newcol_name="Index",
    newcol_func=lambda df: list(range(len(df))),
    newcol_type=int,
    newcol_idx=0,
    keep_new_col=True
)

# check the new columns are kept
print(metered_model.table_cols("mem"))

==========  Table structure customization  ==========

╭───────┬──────┬────────────┬─────────────┬─────────────┬────────────╮
│ Index │  ID  │ Param Cost │ Buffer Cost │ Output Cost │   Total    │
├───────┼──────┼────────────┼─────────────┼─────────────┼────────────┤
│   0   │  1   │ 76.43 MiB  │  43.12 KiB  │ 119.15 MiB  │ 195.62 MiB │
│   1   │ 1.1  │   7 KiB    │      -      │  12.25 MiB  │ 12.26 MiB  │
│   2   │ 1.2  │   512 B    │    520 B    │  12.25 MiB  │ 12.25 MiB  │
│   3   │ 1.3  │     -      │      -      │      -      │     -      │
│   4   │ 1.4  │ 144.25 KiB │      -      │  12.25 MiB  │ 12.39 MiB  │
│   5   │ 1.5  │   512 B    │    520 B    │  12.25 MiB  │ 12.25 MiB  │
│   6   │ 1.6  │     -      │      -      │      -      │     -      │
│   7   │ 1.7  │     -      │      -      │  3.06 MiB   │  3.06 MiB  │
│   8   │ 1.8  │ 288.50 KiB │      -      │  6.12 MiB   │  6.41 MiB  │
│   9   │ 1.9  │   1 KiB    │  1.01 KiB   │  6.12 MiB   │  6.13 MiB  │
│  10   │ 1.10 │     -      │      -      │      -      │     -      │
│  11   │ 1.11 │ 576.50 KiB │      -      │  6.12 MiB   │  6.69 MiB  │
│  12   │ 1.12 │   1 KiB    │  1.01 KiB   │  6.12 MiB   │  6.13 MiB  │
│  13   │ 1.13 │     -      │      -      │      -      │     -      │
│  14   │ 1.14 │     -      │      -      │  1.53 MiB   │  1.53 MiB  │
│  15   │ 1.15 │  1.13 MiB  │      -      │  3.06 MiB   │  4.19 MiB  │
│  16   │ 1.16 │   2 KiB    │  2.01 KiB   │  3.06 MiB   │  3.07 MiB  │
│  17   │ 1.17 │     -      │      -      │      -      │     -      │
│  18   │ 1.18 │  2.25 MiB  │      -      │  3.06 MiB   │  5.31 MiB  │
│  19   │ 1.19 │   2 KiB    │  2.01 KiB   │  3.06 MiB   │  3.07 MiB  │
│  20   │ 1.20 │     -      │      -      │      -      │     -      │
│  21   │ 1.21 │  2.25 MiB  │      -      │  3.06 MiB   │  5.31 MiB  │
│  22   │ 1.22 │   2 KiB    │  2.01 KiB   │  3.06 MiB   │  3.07 MiB  │
│  23   │ 1.23 │     -      │      -      │      -      │     -      │
│  24   │ 1.24 │  2.25 MiB  │      -      │  3.06 MiB   │  5.31 MiB  │
│  25   │ 1.25 │   2 KiB    │  2.01 KiB   │  3.06 MiB   │  3.07 MiB  │
│  26   │ 1.26 │     -      │      -      │      -      │     -      │
│  27   │ 1.27 │     -      │      -      │   784 KiB   │  784 KiB   │
│  28   │ 1.28 │  4.50 MiB  │      -      │  1.53 MiB   │  6.03 MiB  │
│  29   │ 1.29 │   4 KiB    │  4.01 KiB   │  1.53 MiB   │  1.54 MiB  │
│  30   │ 1.30 │     -      │      -      │      -      │     -      │
│  31   │ 1.31 │  9.00 MiB  │      -      │  1.53 MiB   │ 10.53 MiB  │
│  32   │ 1.32 │   4 KiB    │  4.01 KiB   │  1.53 MiB   │  1.54 MiB  │
│  33   │ 1.33 │     -      │      -      │      -      │     -      │
│  34   │ 1.34 │  9.00 MiB  │      -      │  1.53 MiB   │ 10.53 MiB  │
│  35   │ 1.35 │   4 KiB    │  4.01 KiB   │  1.53 MiB   │  1.54 MiB  │
│  36   │ 1.36 │     -      │      -      │      -      │     -      │
│  37   │ 1.37 │  9.00 MiB  │      -      │  1.53 MiB   │ 10.53 MiB  │
│  38   │ 1.38 │   4 KiB    │  4.01 KiB   │  1.53 MiB   │  1.54 MiB  │
│  39   │ 1.39 │     -      │      -      │      -      │     -      │
│  40   │ 1.40 │     -      │      -      │   392 KiB   │  392 KiB   │
│  41   │ 1.41 │  9.00 MiB  │      -      │   392 KiB   │  9.38 MiB  │
│  42   │ 1.42 │   4 KiB    │  4.01 KiB   │   392 KiB   │ 400.01 KiB │
│  43   │ 1.43 │     -      │      -      │      -      │     -      │
│  44   │ 1.44 │  9.00 MiB  │      -      │   392 KiB   │  9.38 MiB  │
│  45   │ 1.45 │   4 KiB    │  4.01 KiB   │   392 KiB   │ 400.01 KiB │
│  46   │ 1.46 │     -      │      -      │      -      │     -      │
│  47   │ 1.47 │  9.00 MiB  │      -      │   392 KiB   │  9.38 MiB  │
│  48   │ 1.48 │   4 KiB    │  4.01 KiB   │   392 KiB   │ 400.01 KiB │
│  49   │ 1.49 │     -      │      -      │      -      │     -      │
│  50   │ 1.50 │  9.00 MiB  │      -      │   392 KiB   │  9.38 MiB  │
│  51   │ 1.51 │   4 KiB    │  4.01 KiB   │   392 KiB   │ 400.01 KiB │
│  52   │ 1.52 │     -      │      -      │      -      │     -      │
│  53   │ 1.53 │     -  

('Index', 'ID', 'Param Cost', 'Buffer Cost', 'Output Cost', 'Total')

In [13]:
# 6.3 Programmable tabular report
cfg.restore() # restore all customized settings
cfg.render_interval = 0

def newcol_logic(df):
    num_col = df['Number']
    return num_col.map_elements(
        lambda x: f"{100 * x / metered_model.param.TotalNum:.4f} %",
        return_dtype=str
    )

print("="*10, " Programmable tabular report ", "="*10)
origin_col = metered_model.table_cols('param')
print(f"origin cols: {origin_col}")

tb, data = metered_model.profile(
    'param', 
    no_tree = True,
    exclude_cols=["Operation_Name"],
    custom_cols={"Operation_Id": 'ID', 
                 "Param_Name": 'Param Name', 
                 "Requires_Grad": 'Trainable',
                 "Numeric_Num": "Number"},
    newcol_name='Percentage',
    newcol_func=newcol_logic,
    newcol_type=str
)

==========  Programmable tabular report  ==========

origin cols: ('Operation_Id', 'Operation_Name', 'Operation_Type', 'Param_Name', 'Requires_Grad', 'Numeric_Num')

╭──────┬───────────────────┬────────────┬───────────┬──────────┬────────────╮
│  ID  │  Operation_Type   │ Param Name │ Trainable │  Number  │ Percentage │
├──────┼───────────────────┼────────────┼───────────┼──────────┼────────────┤
│  1   │    Sequential     │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.1  │      Conv2d       │   weight   │   False   │  1.73 K  │  0.0012 %  │
│ 1.1  │      Conv2d       │    bias    │   False   │  64.00   │  0.0000 %  │
│ 1.2  │    BatchNorm2d    │   weight   │   False   │  64.00   │  0.0000 %  │
│ 1.2  │    BatchNorm2d    │    bias    │   False   │  64.00   │  0.0000 %  │
│ 1.3  │       ReLU        │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.4  │      Conv2d       │   weight   │   False   │ 36.86 K  │  0.0257 %  │
│ 1.4  │      Conv2d       │    bias    │   False   │  64.00   │  0.0000 %  │
│ 1.5  │    BatchNorm2d    │   weight   │   False   │  64.00   │  0.0000 %  │
│ 1.5  │    BatchNorm2d    │    bias    │   False   │  64.00   │  0.0000 %  │
│ 1.6  │       ReLU        │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.7  │     MaxPool2d     │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.8  │      Conv2d       │   weight   │   False   │ 73.73 K  │  0.0513 %  │
│ 1.8  │      Conv2d       │    bias    │   False   │  128.00  │  0.0001 %  │
│ 1.9  │    BatchNorm2d    │   weight   │   False   │  128.00  │  0.0001 %  │
│ 1.9  │    BatchNorm2d    │    bias    │   False   │  128.00  │  0.0001 %  │
│ 1.10 │       ReLU        │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.11 │      Conv2d       │   weight   │   False   │ 147.46 K │  0.1026 %  │
│ 1.11 │      Conv2d       │    bias    │   False   │  128.00  │  0.0001 %  │
│ 1.12 │    BatchNorm2d    │   weight   │   False   │  128.00  │  0.0001 %  │
│ 1.12 │    BatchNorm2d    │    bias    │   False   │  128.00  │  0.0001 %  │
│ 1.13 │       ReLU        │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.14 │     MaxPool2d     │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.15 │      Conv2d       │   weight   │   False   │ 294.91 K │  0.2053 %  │
│ 1.15 │      Conv2d       │    bias    │   False   │  256.00  │  0.0002 %  │
│ 1.16 │    BatchNorm2d    │   weight   │   False   │  256.00  │  0.0002 %  │
│ 1.16 │    BatchNorm2d    │    bias    │   False   │  256.00  │  0.0002 %  │
│ 1.17 │       ReLU        │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.18 │      Conv2d       │   weight   │   False   │ 589.82 K │  0.4105 %  │
│ 1.18 │      Conv2d       │    bias    │   False   │  256.00  │  0.0002 %  │
│ 1.19 │    BatchNorm2d    │   weight   │   False   │  256.00  │  0.0002 %  │
│ 1.19 │    BatchNorm2d    │    bias    │   False   │  256.00  │  0.0002 %  │
│ 1.20 │       ReLU        │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.21 │      Conv2d       │   weight   │   False   │ 589.82 K │  0.4105 %  │
│ 1.21 │      Conv2d       │    bias    │   False   │  256.00  │  0.0002 %  │
│ 1.22 │    BatchNorm2d    │   weight   │   False   │  256.00  │  0.0002 %  │
│ 1.22 │    BatchNorm2d    │    bias    │   False   │  256.00  │  0.0002 %  │
│ 1.23 │       ReLU        │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.24 │      Conv2d       │   weight   │   False   │ 589.82 K │  0.4105 %  │
│ 1.24 │      Conv2d       │    bias    │   False   │  256.00  │  0.0002 %  │
│ 1.25 │    BatchNorm2d    │   weight   │   False   │  256.00  │  0.0002 %  │
│ 1.25 │    BatchNorm2d    │    bias    │   False   │  256.00  │  0.0002 %  │
│ 1.26 │       ReLU        │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.27 │     MaxPool2d     │     -      │     -     │   0.00   │  0.0000 %  │
│ 1.28 │      Conv2d       │   weight   │   False   │  1.18 M  │  0.8210 %  │
│ 1.28 │      Conv2d       │    bias    │   False   │  512.00  │  0.0004 %  │
│ 1.29 │    BatchNorm2d    │   weight   │   False   │  512.00  │  0.0004 %  │
│ 1.29 │    BatchNorm2d    │    bias    │   False   │  512.00  │  0.0004 %  │
│ 1.30 │       ReLU   

In [14]:
# 6.4 tabular report export
print("="*10, " Tabular report export ", "="*10)
tb, data = metered_model.profile(
    'param', 
    show=False,
    no_tree = True,
    exclude_cols=["Operation_Name"],
    custom_cols={"Operation_Id": 'ID', 
                 "Param_Name": 'Param Name', 
                 "Requires_Grad": 'Trainable',
                 "Numeric_Num": "Number"},
    newcol_name='Percentage',
    newcol_func=newcol_logic,
    newcol_type=str,
    save_to='./param_report.xlsx' # or csv
)

==========  Tabular report export  ==========

Param data saved to /home/hzy/project/TorchMeter/examples/param_report.xlsx

In [15]:
# 7. Cross-Platform Support
print("="*10, " Cross-Platform Support ", "="*10)

metered_model.to("cpu")
print(metered_model.device)

if torch.cuda.is_available():
    metered_model.device = "cuda:0"
    print(metered_model.device)

==========  Cross-Platform Support  ==========

cpu

cuda:0

In [16]:
# 8. Model summary

print("="*10, " Model Information ", "="*10)
print(metered_model.model_info)

==========  Model Information  ==========

• Model    : VGG
• Device   : cuda:0
• Signature: forward(self, x)
• Input    : 
   x = Shape([1, 3, 224, 224]) <Tensor>

In [17]:
# 9. Statistics Overview
print("="*10, " Statistics Overview ", "="*10)
print(metered_model.overview())

print("="*10, " Statistics Overview (no warnings) ", "="*10)
print(metered_model.overview(show_warning=False))

print("="*10, " Statistics Overview (custom) ", "="*10)
print(metered_model.overview("param", "mem"))

==========  Statistics Overview  ==========

Warming Up: 100%|██████████| 10/10 [00:00<00:00, 423.15it/s]


Benchmark Inference Time & Throughput: 100%|██████████| 1280/1280 [00:00<00:00, 4652.85module/s]


 ────────────── Model INFO ───────────────                  ───────────── Param INFO ─────────────      
             • Model    : VGG                                           • Statistics: param                        
             • Device   : cuda:0                                        • Learnable Parameters Num: 123.64 M       
             • Signature: forward(self, x)                              • Total Parameters Num    : 143.68 M       
             • Input    :                                              ──────────────────────────────────────      
                x = Shape([1, 3, 224, 224]) <Tensor>                                                               
            ─────────────────────────────────────────                                                              
 ────────────────────────── Cal INFO ──────────────────────────    ────────────────── Mem INFO ─────────────────── 
  • Statistics: cal                                                 • Statistics: mem                              
  • FLOPs               : 39.34 G                                   • Parameters Memory Cost: 548.09 MiB, 82.12 %  
  • MACs(aka MACC, MADD): 19.68 G                                   • Buffers Memory Cost   : 43.12 KiB, 0.01 %    
  ⚠  Warning: the result may be inaccurate, cause:                  • FeatureMap Memory Cost: 119.31 MiB, 17.88 %  
    ▶  Some modules don't support calculation measurement yet.      • Total Memory Cost     : 667.44 MiB           
    ☑  use `Meter(your_model).profile('cal')` to see more.         ─────────────────────────────────────────────── 
 ──────────────────────────────────────────────────────────────                                                    
           ──────────────── Ittp INFO ────────────────                                                             
            • Statistics: ittp                                                                                     
            • Benchmark Times: 20                                                                                  
            • Inference Elapse: 2.72 ms ± 15.71 us                                                                 
            • Throughput      : 367.96 IPS ± 2.12 IPS                                                              
           ─────────────────────────────────────────── 

==========  Statistics Overview (no warnings)  ==========

Benchmark Inference Time & Throughput: 100%|██████████| 1280/1280 [00:00<00:00, 4717.00module/s]


 ────────────── Model INFO ───────────────                  ───────────── Param INFO ─────────────           
        • Model    : VGG                                           • Statistics: param                             
        • Device   : cuda:0                                        • Learnable Parameters Num: 123.64 M            
        • Signature: forward(self, x)                              • Total Parameters Num    : 143.68 M            
        • Input    :                                              ──────────────────────────────────────           
           x = Shape([1, 3, 224, 224]) <Tensor>                                                                    
       ─────────────────────────────────────────                                                                   
           ─────────── Cal INFO ────────────                  ────────────────── Mem INFO ───────────────────      
            • Statistics: cal                                  • Statistics: mem                                   
            • FLOPs               : 39.34 G                    • Parameters Memory Cost: 548.09 MiB, 82.12 %       
            • MACs(aka MACC, MADD): 19.68 G                    • Buffers Memory Cost   : 43.12 KiB, 0.01 %         
           ─────────────────────────────────                   • FeatureMap Memory Cost: 119.31 MiB, 17.88 %       
                                                               • Total Memory Cost     : 667.44 MiB                
                                                              ───────────────────────────────────────────────      
      ──────────────── Ittp INFO ────────────────                                                                  
       • Statistics: ittp                                                                                          
       • Benchmark Times: 20                                                                                       
       • Inference Elapse: 2.73 ms ± 13.70 us                                                                      
       • Throughput      : 366.32 IPS ± 1.84 IPS                                                                   
      ─────────────────────────────────────────── 

==========  Statistics Overview (custom)  ==========

 ────────────── Model INFO ───────────────                  ───────────── Param INFO ─────────────       
            • Model    : VGG                                           • Statistics: param                         
            • Device   : cuda:0                                        • Learnable Parameters Num: 123.64 M        
            • Signature: forward(self, x)                              • Total Parameters Num    : 143.68 M        
            • Input    :                                              ──────────────────────────────────────       
               x = Shape([1, 3, 224, 224]) <Tensor>                                                                
           ─────────────────────────────────────────                                                               
        ────────────────── Mem INFO ───────────────────                                                            
         • Statistics: mem                                                                                         
         • Parameters Memory Cost: 548.09 MiB, 82.12 %                                                             
         • Buffers Memory Cost   : 43.12 KiB, 0.01 %                                                               
         • FeatureMap Memory Cost: 119.31 MiB, 17.88 %                                                             
         • Total Memory Cost     : 667.44 MiB                                                                      
        ─────────────────────────────────────────────── 

In [18]:
# 10. Advanced Usage

## 10.1 post export of tabular report
print("="*10, " Custom export ", "="*10)
metered_model.table_renderer.export(df=data,
                                    save_path=".",
                                    format="csv",
                                    file_suffix="custom_export",
                                    raw_data=True)

==========  Custom export  ==========

Custom_export data saved to /home/hzy/project/TorchMeter/examples/VGG_custom_export.csv

In [19]:
## 10.2 repeat footer
import torch.nn as nn
from random import sample

print("="*10, " Custom repeat block footer ", "="*10)

class RepeatModel(nn.Module):
    def __init__(self, repeat_winsz:int=1, repeat_time:int=2):
        super(RepeatModel, self).__init__()
        
        layer_candidates = [nn.Linear(10, 10), 
                            nn.ReLU(),
                            nn.Identity()]

        pick_modules = sample(layer_candidates, repeat_winsz)
        all_modules = pick_modules * repeat_time

        self.layers = nn.ModuleList(all_modules)

metered_model = Meter(RepeatModel(repeat_winsz=2, repeat_time=3), 
                      device="cpu")

### 10.2.1 change with a hard-coding string 
print("-"*10, " Using hard-coding string ", "-"*10)
metered_model.tree_renderer.repeat_footer = "My custom footer"
print(metered_model.structure)

==========  Custom repeat block footer  ==========

Finish Scanning model in 0.0022 seconds

----------  Using hard-coding string  ----------

RepeatModel
└── (1) layers ModuleList
    └── ┏━━ Repeat [3] Times ━━┓
        ┃ (1.x) 0 Identity     ┃
        ┃ (1.(x+1)) 1 ReLU     ┃
        ┃ -------------------- ┃
        ┃ My custom footer     ┃
        ┗━━━━━━━━━━━━━━━━━━━━━━┛

In [20]:
### 10.2.2 change with a string which access operation node attributes
print("-"*10, " Using dynamic string with attributes resolved ", "-"*10)
metered_model.tree_renderer.repeat_footer = "This module type is <type>"
print(metered_model.structure)

----------  Using dynamic string with attributes resolved  ----------

RepeatModel
└── (1) layers ModuleList
    └── ┏━━━━━━ Repeat [3] Times ━━━━━━┓
        ┃ (1.y) 0 Identity             ┃
        ┃ (1.(y+1)) 1 ReLU             ┃
        ┃ ---------------------------- ┃
        ┃ This module type is Identity ┃
        ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

In [21]:
### 10.2.3 change with a function which accept a attr-dictionary and return a string
print("-"*10, " Using funtion ", "-"*10)
def my_footer(attr_dict):
    repeat_win_size = attr_dict["repeat_winsz"]
    if repeat_win_size > 1:
        return f"There are {repeat_win_size} modules in a repeat window"
    else:
        return "The repeat window only contains one module"
metered_model.tree_renderer.repeat_footer = my_footer
print(metered_model.structure)

----------  Using funtion  ----------

RepeatModel
└── (1) layers ModuleList
    └── ┏━━━━━━━━━━━ Repeat [3] Times ━━━━━━━━━━━┓
        ┃ (1.i) 0 Identity                       ┃
        ┃ (1.(i+1)) 1 ReLU                       ┃
        ┃ -------------------------------------- ┃
        ┃ There are 2 modules in a repeat window ┃
        ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

In [22]:
## 10.3 config management

print("="*10, " Efficient config management ", "="*10)
### 10.3.1 show config
print("-"*10, " config display ", "-"*10)
print(cfg)

==========  Efficient config management  ==========

----------  config display  ----------

• Config file: None(default setting below)

• render_interval: 0 | <int>

• tree_fold_repeat: True | <bool>

• tree_repeat_block_args: namespace{
│   title = [i]Repeat [[b]<repeat_time>[/b]] Times[/] | <str>
│   title_align = center | <str>
│   subtitle = None | <NoneType>
│   subtitle_align = center | <str>
│   style = dark_goldenrod | <str>
│   highlight = True | <bool>
│   box = HEAVY_EDGE | <str>
│   border_style = dim | <str>
│   width = None | <NoneType>
│   height = None | <NoneType>
│   padding = list(
│   │   - 0 | <int>
│   │   - 1 | <int>
│   └─  )
│   expand = False | <bool>
└─  }

• tree_levels_args: namespace{
│   default = namespace{
│   │   label = [b gray35](<node_id>) [green]<name>[/green] [cyan]<type>[/] | <str>
│   │   style = tree | <str>
│   │   guide_style = light_coral | <str>
│   │   highlight = True | <bool>
│   │   hide_root = False | <bool>
│   │   expanded = True | <bool>
│   └─  }
│   0 = namespace{
│   │   label = [b light_coral]<name>[/] | <str>
│   │   guide_style = light_coral | <str>
│   └─  }
└─  }

• table_column_args: namespace{
│   style = none | <str>
│   justify = center | <str>
│   vertical = middle | <str>
│   overflow = fold | <str>
│   no_wrap = False | <bool>
└─  }

• table_display_args: namespace{
│   style = spring_green4 | <str>
│   highlight = True | <bool>
│   width = None | <NoneType>
│   min_width = None | <NoneType>
│   expand = False | <bool>
│   padding = list(
│   │   - 0 | <int>
│   │   - 1 | <int>
│   └─  )
│   collapse_padding = False | <bool>
│   pad_edge = True | <bool>
│   leading = 0 | <int>
│   title = None | <NoneType>
│   title_style = bold | <str>
│   title_justify = center | <str>
│   caption = None | <NoneType>
│   caption_style = None | <NoneType>
│   caption_justify = center | <str>
│   show_header = True | <bool>
│   header_style = bold | <str>
│   show_footer = False | <bool>
│   footer_style = italic | <str>
│   show_lines = False | <bool>
│   row_styles = None | <NoneType>
│   show_edge = True | <bool>
│   box = ROUNDED | <str>
│   safe_box = True | <bool>
│   border_style = None | <NoneType>
└─  }

• combine: namespace{
│   horizon_gap = 2 | <int>
└─  }

In [23]:
### 10.3.2 retrieve config setting
print("-"*10, " config settings retrieval ", "-"*10)
print(
    f"config_file: {cfg.config_file}\n",
    f"render time interval: {cfg.render_interval}\n",
    f"tree default guide line style: {cfg.tree_levels_args.default.guide_style}\n",
    f"table col justify: {cfg.table_column_args.justify}\n",
    f"gap between tree and table in profiling: {cfg.combine.horizon_gap}"
)

----------  config settings retrieval  ----------

config_file: None
 render time interval: 0
 tree default guide line style: light_coral
 table col justify: center
 gap between tree and table in profiling: 2

In [24]:
### 10.3.3 change config settings
print("-"*10, " change config settings ", "-"*10)
cfg.render_interval = 0.5
cfg.tree_levels_args.default.guide_style = "buld"
cfg.table_display_args = {
    "show_header": False,
    "show_lines": True
}
print(cfg.render_interval)
print(cfg.tree_levels_args.default.guide_style)
print(cfg.table_display_args.show_header)
print(cfg.table_display_args.show_lines)

----------  change config settings  ----------

0.5

buld

False

True

In [25]:
import os
### 10.3.4 save config
print("-"*10, " dump config settings ", "-"*10)
des = "./my_config.yaml"
cfg.dump(save_path=des)
abs_des = os.path.abspath(des)
if os.path.exists(abs_des):
    print(f"config dumped successfully to {abs_des}")

----------  dump config settings  ----------

config dumped successfully to /home/hzy/project/TorchMeter/examples/my_config.yaml

In [26]:
### 10.3.5 restore config
print("-"*10, " restore all config settings ", "-"*10)
cfg.restore()
print(cfg)

----------  restore all config settings  ----------

• Config file: None(default setting below)

• render_interval: 0.15 | <float>

• tree_fold_repeat: True | <bool>

• tree_repeat_block_args: namespace{
│   title = [i]Repeat [[b]<repeat_time>[/b]] Times[/] | <str>
│   title_align = center | <str>
│   subtitle = None | <NoneType>
│   subtitle_align = center | <str>
│   style = dark_goldenrod | <str>
│   highlight = True | <bool>
│   box = HEAVY_EDGE | <str>
│   border_style = dim | <str>
│   width = None | <NoneType>
│   height = None | <NoneType>
│   padding = list(
│   │   - 0 | <int>
│   │   - 1 | <int>
│   └─  )
│   expand = False | <bool>
└─  }

• tree_levels_args: namespace{
│   default = namespace{
│   │   label = [b gray35](<node_id>) [green]<name>[/green] [cyan]<type>[/] | <str>
│   │   style = tree | <str>
│   │   guide_style = light_coral | <str>
│   │   highlight = True | <bool>
│   │   hide_root = False | <bool>
│   │   expanded = True | <bool>
│   └─  }
│   0 = namespace{
│   │   label = [b light_coral]<name>[/] | <str>
│   │   guide_style = light_coral | <str>
│   └─  }
└─  }

• table_column_args: namespace{
│   style = none | <str>
│   justify = center | <str>
│   vertical = middle | <str>
│   overflow = fold | <str>
│   no_wrap = False | <bool>
└─  }

• table_display_args: namespace{
│   style = spring_green4 | <str>
│   highlight = True | <bool>
│   width = None | <NoneType>
│   min_width = None | <NoneType>
│   expand = False | <bool>
│   padding = list(
│   │   - 0 | <int>
│   │   - 1 | <int>
│   └─  )
│   collapse_padding = False | <bool>
│   pad_edge = True | <bool>
│   leading = 0 | <int>
│   title = None | <NoneType>
│   title_style = bold | <str>
│   title_justify = center | <str>
│   caption = None | <NoneType>
│   caption_style = None | <NoneType>
│   caption_justify = center | <str>
│   show_header = True | <bool>
│   header_style = bold | <str>
│   show_footer = False | <bool>
│   footer_style = italic | <str>
│   show_lines = False | <bool>
│   row_styles = None | <NoneType>
│   show_edge = True | <bool>
│   box = ROUNDED | <str>
│   safe_box = True | <bool>
│   border_style = None | <NoneType>
└─  }

• combine: namespace{
│   horizon_gap = 2 | <int>
└─  }

In [27]:
### 10.3.6 reuse config
print("-"*10, " config reuse ", "-"*10)
new_cfg = get_config(config_path=abs_des)
print(f"reuse: {new_cfg.config_file}")
print(new_cfg)
print("You can compare with the restored settings in last cell, ",
      "to check if the settings before restore are reused.")

----------  config reuse  ----------

reuse: /home/hzy/project/TorchMeter/examples/my_config.yaml

• Config file: /home/hzy/project/TorchMeter/examples/my_config.yaml

• render_interval: 0.5 | <float>

• tree_fold_repeat: True | <bool>

• tree_repeat_block_args: namespace{
│   title = [i]Repeat [[b]<repeat_time>[/b]] Times[/] | <str>
│   title_align = center | <str>
│   subtitle = None | <NoneType>
│   subtitle_align = center | <str>
│   style = dark_goldenrod | <str>
│   highlight = True | <bool>
│   box = HEAVY_EDGE | <str>
│   border_style = dim | <str>
│   width = None | <NoneType>
│   height = None | <NoneType>
│   padding = list(
│   │   - 0 | <int>
│   │   - 1 | <int>
│   └─  )
│   expand = False | <bool>
└─  }

• tree_levels_args: namespace{
│   default = namespace{
│   │   label = [b gray35](<node_id>) [green]<name>[/green] [cyan]<type>[/] | <str>
│   │   style = tree | <str>
│   │   guide_style = buld | <str>
│   │   highlight = True | <bool>
│   │   hide_root = False | <bool>
│   │   expanded = True | <bool>
│   └─  }
│   0 = namespace{
│   │   label = [b light_coral]<name>[/] | <str>
│   │   guide_style = light_coral | <str>
│   └─  }
└─  }

• table_column_args: namespace{
│   style = none | <str>
│   justify = center | <str>
│   vertical = middle | <str>
│   overflow = fold | <str>
│   no_wrap = False | <bool>
└─  }

• table_display_args: namespace{
│   style = spring_green4 | <str>
│   highlight = True | <bool>
│   width = None | <NoneType>
│   min_width = None | <NoneType>
│   expand = False | <bool>
│   padding = list(
│   │   - 0 | <int>
│   │   - 1 | <int>
│   └─  )
│   collapse_padding = False | <bool>
│   pad_edge = True | <bool>
│   leading = 0 | <int>
│   title = None | <NoneType>
│   title_style = bold | <str>
│   title_justify = center | <str>
│   caption = None | <NoneType>
│   caption_style = None | <NoneType>
│   caption_justify = center | <str>
│   show_header = False | <bool>
│   header_style = bold | <str>
│   show_footer = False | <bool>
│   footer_style = italic | <str>
│   show_lines = True | <bool>
│   row_styles = None | <NoneType>
│   show_edge = True | <bool>
│   box = ROUNDED | <str>
│   safe_box = True | <bool>
│   border_style = None | <NoneType>
└─  }

• combine: namespace{
│   horizon_gap = 2 | <int>
└─  }

You can compare with the restored settings in last cell,  to check if the settings before restore are reused.